# Structure of the library
In this notebook the basic components and ideas behind the library will be explained and demonstrated. All exampes are presented for object detection but also extend to instance segmentation.

## Overview
The library consists of three major parts: plotting functions/controlls, datasets/descriptors and dashboards.

Plotting functions and controls provide functions to generate bokeh plots and panel controll elements, these can be used to build the gui interface for the dashboards.
The datasets allow for data (e.g. records) to be consumed by a dashboard. Descriptors are used to modify the behaviour of datassets and dashboards to make them easily adaptable.
Dashboards combine plotting functions, controlls and datasets to generate insight into the data.

### Dashboards
Each dashboard is made up from a gui (based on plotting functions and controlls) and a dataset, which provides the data for the gui.
The abstarct base class for a dashboard only defines an `__init__` function which takes a `width` and `height` parameters and calles the `build_gui` function. Furthermore, the abstact methods `build_gui` and `show` are defined, which need to be implemented by all inherited classes. `build_gui` is used to build the elements that make up the dashboard, it should not return anything but set the class attributes that make up the gui. `show` should return panel object that can be displayed.
Because the `__init__` function automatically calls `build_gui` the super method for the init should be called last.

To show a simple form of a dashboard we will create a dashboard below, that inherits from the Dashboard class and displays a given string with a lightgray background.

In [1]:
import panel as pn
from icevision_dashboards.core.dashboards import Dashboard

In [2]:
# make sure panel output will be diplayed in the notebook
pn.extension()

In [3]:
# very simpel dashboard
class SimpelDashboard(Dashboard):
    # define a __init__ function that takes a string which will be displayed
    def __init__(self, string, width=100, height=100):
        self.string = string
        # calling super will set the width and height attribute for the dashbaord and call the build_gui function
        super().__init__(width, height)
        
    def build_gui(self):
        # build a simple gui that takes the classes string and displayes as panel.pane.Markdown with a lightgray background
        self.gui = pn.pane.Markdown(self.string, background="lightgray")
        
    def show(self):
        # just return self.gui to display the build gui
        return self.gui

In [4]:
SimpelDashboard("This is a test string").show()

Markdown(str, background='lightgray')

Many dashboards in the library use descriptors for customizability. Before we take a look at how dashboards use descriptors and how to simply change dashboards with descriptors first a bit about data in IcevisionDashboards.

### Data
Datasets in IcevisionDashboards (not to confuse with datasets from icevision) consist of some data (often a list of records) and some descriptors (for more on descriptors in Python see here: https://docs.python.org/3/howto/descriptor.html). 

IcevisionDashboars defines a DatasetDescriptor: 

```python
class DatasetDescriptor(ABC):
    """Abstract base class for descriptors of datasets.
    The private name of the descriptor is the defined name with a prefix _.
    The __get__ function will call the calculate_description function if the value of the descriptor is None and then return the value else it will just return the value of the descriptor.
    The __set__ function only allows for the attribute to be set to None, which will trigger a recomputation the next time the __get__ function is called.
    When inheriting this class the function calculate_description needs to be implemented, which defines how the private value should be calculated.
    """
    def __set_name__(self, owner, name):
        owner._descriptors.append(self)
        self.private_name = '_' + name

    def __get__(self, obj, objtype=None):
        "Attribute will be recomputed if it is None else the befor computed version will be returned."
        if getattr(obj, self.private_name) is None:
            value = self.calculate_description(obj)
            setattr(obj, self.private_name, value)
        return getattr(obj, self.private_name)

    def __set__(self, obj, value):
        "Attribute can only be set to None externaly, otherwise an ValueError will be raised."
        if value is None:
            setattr(obj, self.private_name, value)
        else:
            raise ValueError("Attribute can only be set to None externaly.")
            
    @abstractmethod
    def calculate_description(self, obj):
        pass
```

The private name is always the name of the descriptor with prefix `_`. The idea for the DatasetDescriptors is, that it uses the `base_data` or other data of the dataset and calculates new informations based on that. For that reason, the `__set__` function only allows for a descriptor to be set to None, which will trigger a recomputation the next time the `__get__` function is called. The `__get__` automatically check if the value of the private name is None and if so calls the `calculate_description` to recompute the value of the descriptor or otherwise return the current value of the descriptor.

    Note: The DatasetDescriptor automatically registers the descriptor in the _descriptor list of the owner. The Dataset class already has the _descriptor attribute as a class attribute. This allows for a dataset to reset all descriptors if the base data changes.

The reason behind this behaviour is to allow for datasets/dashboard stats to be changed dynamically. For example when creating new dataset with a dashboard the stats of the new dataset need to be calculated after each change of the dataset.

Most descriptors (based on DatasetDescriptor) return a pandas dataframe for `calculate_description` that is than cosumed by a dashboard.

For example the `BboxRecordDataset` uses the following descriptors:

    - data = DataDescriptorBbox()
    - gallery_data = GalleryStatsDescriptorBbox()
    - stats_dataset = StatsDescriptorBbox()
    - stats_image = ImageStatsDescriptorBbox()
    - stats_class = ClassStatsDescriptorBbox()
    - stats = StatsDescriptorBbox()
   
In the example below we create a regular `BboxRecordDataset` and then create our own with a custom desriptor for stats.

First we load the records for the fridge dataset and create a `BboxRecordDataset` from the records.

In [5]:
import icedata
from icevision.data.data_splitter import RandomSplitter
# Load the Fridge dataset
path = icedata.fridge.load_data()
# get the class map
class_map = icedata.fridge.class_map()
# parse the data
parser = icedata.fridge.parser(data_dir=path)
# we just want to have a look at the data so we don't split the data
records = parser.parse(RandomSplitter([1]))[0]

  0%|          | 0/128 [00:00<?, ?it/s]

INFO     - Autofixing records | icevision.parsers.parser:parse:136


  0%|          | 0/128 [00:00<?, ?it/s]

In [6]:
from icevision_dashboards.data import BboxRecordDataset

In [35]:
# Create BboxRecordDataset for the records and display the stats discriptor
fridge_bbox_record_dataset = BboxRecordDataset(records)
fridge_bbox_record_dataset.stats

,no_imgs,no_classes,classes,area_min,area_max,num_annotations_min,num_annotations_max,name,description
0,122,4,"[2, 4, 3, 1]",11771,105651,1,4,None,None


Define a custom descriptor for stats. We will base the data for the descriptor on the base_data (the list of records). The `base_data` attribute is always set to the input data of the `__init__` function.

In [36]:
import pandas as pd
from icevision_dashboards.core.data import DatasetDescriptor

In [37]:
class CustomStatsDescriptor(DatasetDescriptor):
    # We only need to define the calculate_description function the getter and setter are already defined (for more information check the core.data.ipynb in the nbs folder)
    def calculate_description(self, obj):
        return pd.DataFrame([{"num_records": len(obj.base_data)}])

With the new descriptor we can define a new CustomBboxRecordDataset that uses our CustomStatsDescriptor.

In [38]:
class CustomBboxRecordDataset(BboxRecordDataset):
    stats = CustomStatsDescriptor()

In [39]:
custom_bbox_record_datset = CustomBboxRecordDataset(records)
custom_bbox_record_datset.stats

,num_records
0,122


The other descriptors from above are still the ones defined in the BboxRecordDataset class.

In [40]:
custom_bbox_record_datset.stats_class

,Class,imgs,objects,avg_objects_per_img,frac_of_labels
0,1,81.0,81.0,1.0,0.25
1,2,82.0,82.0,1.0,0.26
2,3,79.0,79.0,1.0,0.25
3,4,79.0,79.0,1.0,0.25


Because the BboxRecordDataset automatically converts the list of records into a ObservableList (for more on that [here](../nbs/core.data.ipynb)) if we now change the observable list of records the canges will be automatically reflected by the descriptor.

In [42]:
custom_bbox_record_datset.base_data = custom_bbox_record_datset.base_data + custom_bbox_record_datset.base_data
custom_bbox_record_datset.stats

,num_records
0,242


### Plotting functions
The plotting functions supplied by IcevisionDashboards are mostly base on bokeh and panel and provide different levels of abstactions for creating plots and controll elements.

Many basic plotting functions like barplot are wrapper around bokeh that extend the functionality to allow for creating multiple plots with one line of code:

In [45]:
from icevision_dashboards.plotting.core import barplot
import numpy as np

bar_plot_data_1_counts = np.array([10, 15, 20])
bar_plot_data_1_values = np.array(["a", "b", "c"])

bar_plot_data_2_counts = np.array([15, 5, 25])
bar_plot_data_2_values = np.array(["a", "b", "c"])

In [46]:
# create a barplot with linked axis (both plots have the same axis height and width)
p = barplot([bar_plot_data_1_counts, bar_plot_data_2_counts], [bar_plot_data_1_values, bar_plot_data_2_values], bar_type="vertical")
# if the input for counts and values is a list the function returns a list of plots
pn.Row(*p)

Row
    [0] Bokeh(Figure)
    [1] Bokeh(Figure)

In [47]:
# same as above but without linking the axis
p = barplot([bar_plot_data_1_counts, bar_plot_data_2_counts], [bar_plot_data_1_values, bar_plot_data_2_values], linked_axis=False, bar_type="vertical")
pn.Row(*p)

Row
    [0] Bokeh(Figure)
    [1] Bokeh(Figure)

## Recap

Icevision Dashboards consists of three major parts:

    - Plotting functions: To create plots and controll elements
    - Dataset and descriptors: To make the data (records) consumalbel for the dashboards and provide was to extend/change dashboards in a simple manner
    - Dashboards: Combine plotting functions, datasets and descriptors to provide insight into the data
    
Above we saw some examples of how to interact with, use and combine the different components. For further examples take a look at the following notebooks:

[Custom Dashboards](./Custom_dashboards.ipynb)
    
If you want to see how to use the existing dashboards and dataset to support you with your work take a look at the following notebooks:

[Object detection](./object_detection_fridge_dataset.ipynb)

[Instance Segmentation](./instance_segmentation_pennfudan_dataset.ipynb)

[Creating a new dataset](./Creating_a_new_dataset.ipynb)